<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Submit-DL-preds" data-toc-modified-id="Submit-DL-preds-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Submit DL preds</a></span></li><li><span><a href="#Load-lgbm-preds" data-toc-modified-id="Load-lgbm-preds-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Load lgbm preds</a></span></li><li><span><a href="#Ridge-regression-of-lgbm-&amp;-dl-preds" data-toc-modified-id="Ridge-regression-of-lgbm-&amp;-dl-preds-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Ridge regression of lgbm &amp; dl preds</a></span></li><li><span><a href="#Load-XGboost-preds" data-toc-modified-id="Load-XGboost-preds-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Load XGboost preds</a></span></li><li><span><a href="#Stack-lgbm,-dl,-xgboost" data-toc-modified-id="Stack-lgbm,-dl,-xgboost-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Stack lgbm, dl, xgboost</a></span></li></ul></div>

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 4
%autosave 120

Autosaving every 120 seconds


In [2]:
from fastai.io import *
from fastai.structured import *
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from pandas_summary import DataFrameSummary
from IPython.display import display
from sklearn import metrics
import feather
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import mean_squared_error
import warnings
import gc
from sklearn.linear_model import Ridge
import xgboost as xgb

In [8]:
train_df2 = feather.read_dataframe('train_df_alpha')
test_df2 = feather.read_dataframe('test_df_alpha')

/home/chandrasekhar/anaconda3/envs/fastai/lib/python3.6/site-packages/pyarrow/pandas_compat.py:752: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels


In [17]:
target = train_df2['target']

### Submit DL preds

In [3]:
oof_dl = np.load('oof_dl.npy')
predictions_dl = np.load('predictions_dl.npy')

In [4]:
oof_dl

array([-0.52861,  0.30983,  0.7462 , ..., -0.39692, -3.28557, -0.75396])

In [5]:
predictions_dl

array([-1.86731, -0.53013, -0.85165, ...,  0.39517, -3.64371,  0.1123 ])

In [7]:
len(oof_dl), len(predictions_dl)

(201917, 123623)

In [9]:
train_df2.shape, test_df2.shape

((201917, 386), (123623, 384))

In [14]:
sum(train_df2.target.values)

-79481.86060092675

In [12]:
sum(predictions_dl)

-49717.78315293922

In [8]:
np.sqrt(mean_squared_error(target.values, oof_dl))

3.6963961816811954

In [5]:
submission_name = pd.to_datetime("today")

In [6]:
sub_df = pd.DataFrame({"card_id":test_df2["card_id"].values})
sub_df["target"] = predictions_dl
sub_df.to_csv(f'dl_submission-{submission_name}.csv', index=False)

In [7]:
filename = f'dl_submission-{submission_name}.csv'

In [8]:
filename

'dl_submission-2019-02-22 21:30:24.851272.csv'

In [9]:
!kaggle competitions submit elo-merchant-category-recommendation -f 'dl_submission-2019-02-22 21:30:24.851272.csv' -m "dl with 178 cols"

100%|#######################################| 4.24M/4.24M [00:07<00:00, 592kB/s]
Successfully submitted to Elo Merchant Category Recommendation

### Load lgbm preds

In [15]:
oof_lgbm = np.load('oof_lgbm.npy')
predictions_lgbm = np.load('predictions_lgbm.npy')

In [18]:
np.sqrt(mean_squared_error(target.values, oof_lgbm))

3.6437856343297264

In [19]:
predictions_lgbm.shape, predictions_dl.shape

((123623,), (123623,))

In [20]:
oof_lgbm.shape, oof_dl.shape

((201917,), (201917,))

### Ridge regression of lgbm & dl preds

In [11]:
train_stack = np.vstack([oof_lgbm, oof_dl]).transpose()
test_stack = np.vstack([predictions_lgbm, predictions_dl]).transpose()

folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=15)
oof_stacked = np.zeros(train_stack.shape[0])
predictions_stacked = np.zeros(test_stack.shape[0])

for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_stack, train_df2['outliers'].values)):
    print("fold n°{}".format(fold_))
    trn_data, trn_y = train_stack[trn_idx], target.iloc[trn_idx].values
    val_data, val_y = train_stack[val_idx], target.iloc[val_idx].values

    clf = Ridge(alpha=1)
    clf.fit(trn_data, trn_y)
    
    oof_stacked[val_idx] = clf.predict(val_data)
    predictions_stacked += clf.predict(test_stack) / folds.n_splits



np.sqrt(mean_squared_error(target.values, oof_stacked))

fold n°0
fold n°1
fold n°2
fold n°3
fold n°4


3.643089002165104

### Load XGboost preds

In [12]:
oof_xgb_3_2 = np.load('oof_xgb_3_2.npy')
predictions_xgb_3_2 = np.load('predictions_xgb_3_2.npy')

In [13]:
np.sqrt(mean_squared_error(target.values, oof_xgb_3_2))

3.649844028552147

### Stack lgbm, dl, xgboost

In [14]:
train_stack = np.vstack([oof_lgbm, oof_dl, oof_xgb_3_2]).transpose()
test_stack = np.vstack([predictions_lgbm, predictions_dl, predictions_xgb_3_2]).transpose()

folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=15)
oof_stacked = np.zeros(train_stack.shape[0])
predictions_stacked = np.zeros(test_stack.shape[0])

for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_stack, train_df2['outliers'].values)):
    print("fold n°{}".format(fold_))
    trn_data, trn_y = train_stack[trn_idx], target.iloc[trn_idx].values
    val_data, val_y = train_stack[val_idx], target.iloc[val_idx].values

    clf = Ridge(alpha=1)
    clf.fit(trn_data, trn_y)
    
    oof_stacked[val_idx] = clf.predict(val_data)
    predictions_stacked += clf.predict(test_stack) / folds.n_splits



np.sqrt(mean_squared_error(target.values, oof_stacked))

fold n°0
fold n°1
fold n°2
fold n°3
fold n°4


3.641238686022574

1. Round 1
    - XGB - 3.649844028552147
    - LGB - 3.6437856343297264 
    - Stacked - 3.6416503351286953
1. Round 2
    - DL - 3.6963961816811954
    - LGB - 3.6437856343297264 
    - Stacked - 3.643089002165104
1. Round 3
    - DL - 3.6963961816811954
    - LGB - 3.6437856343297264 
    - XGB - 3.649844028552147
    - Stacked - 3.641238686022574

In [18]:
submission_name = pd.to_datetime("today")
sub_df = pd.DataFrame({"card_id":test_df2["card_id"].values})
sub_df["target"] = predictions_stacked
sub_df.to_csv(f'dl_xgb_submission_stacked_lgb-{submission_name}.csv', index=False)
filename = f'dl_xgb_submission_stacked_lgb-{submission_name}.csv'

In [19]:
filename

'dl_xgb_submission_stacked_lgb-2019-02-26 22:45:24.597929.csv'

In [20]:
!kaggle competitions submit elo-merchant-category-recommendation -f 'dl_xgb_submission_stacked_lgb-2019-02-26 22:45:24.597929.csv' -m "dl & xgb & lgbm stacked with 178 cols - 3.641238686022574"

100%|#######################################| 4.23M/4.23M [00:22<00:00, 196kB/s]
Successfully submitted to Elo Merchant Category Recommendation